In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec



import matplotlib.pyplot as plt # data visualisation
import seaborn as sns  # data visualisation
%matplotlib inline
import os


import torch
from transformers import BertTokenizer, BertModel


import numpy as np

# Loading and Preparing Data

In [5]:
CBD = pd.read_csv('/content/drive/MyDrive/448 - Project/CollegeBasketballPlayers2009-2021.csv') 
TextAndNBA = pd.read_csv('/content/drive/MyDrive/448 - Project/Tdata/TextPlusNBA.csv')  

<ipython-input-5-39e733ad5d32>:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  CBD = pd.read_csv('/content/drive/MyDrive/448 - Project/CollegeBasketballPlayers2009-2021.csv')


In [6]:
CBD = CBD.dropna()

CBD = CBD.drop(['Unnamed: 64', 'Unnamed: 65'], axis=1)

CBD['GP'] = CBD['GP'].astype(int)
CBD['Min_per'] = CBD['Min_per'].astype(float)

In [7]:
CBD.head()

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,mp,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts
4619,Vernon Macklin,Florida,SEC,34,62.5,116.3,19.8,60.7,60.85,11.9,...,25.2647,3.802010,1.05341,2.6765,2.8529,5.5294,0.7647,0.5882,0.8529,10.5588
4648,Ryan Reid,Florida St.,ACC,31,54.8,97.5,18.3,49.1,51.58,9.9,...,22.7097,0.514712,2.20601,1.8710,2.0968,3.9677,1.1290,0.5806,0.5161,6.7742
4649,Kyle Singler,Duke,ACC,40,89.9,117.4,24.0,49.2,54.49,7.0,...,35.9500,7.248220,2.69598,2.2500,4.7500,7.0000,2.4000,1.0250,0.8000,17.6750
4650,Nolan Smith,Duke,ACC,38,84.4,115.5,23.7,49.6,53.44,2.9,...,35.5526,5.382730,2.30597,0.9211,1.8684,2.7895,3.0000,1.1842,0.2368,17.3684
4652,E'Twaun Moore,Purdue,B10,35,78.5,103.9,27.7,49.9,53.13,3.4,...,31.5143,4.757390,2.99876,0.9143,2.9143,3.8286,2.6571,1.4571,0.2857,16.4286


In [8]:
CBD.columns

Index(['player_name', 'team', 'conf', 'GP', 'Min_per', 'Ortg', 'usg', 'eFG',
       'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA',
       'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'Rec Rank', 'ast/tov', 'rimmade',
       'rimmade+rimmiss', 'midmade', 'midmade+midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade',
       'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts'],
      dtype='object')

In [9]:
CBD = CBD.drop(['year'], axis=1)

In [10]:
TextAndNBA.columns

Index(['name', 'year', 'text', 'link', 'QA_pair_count', 'Sentence Count',
       'Common Word Count', 'No Special Char', 'preprocessed_text',
       'Word Count', 'Preprocessed Common Word Count', 'replaced_text',
       'removed_text', 'Number UNK', 'Total Words', 'Percent UNK',
       'TF_IDF_SUM', 'TF_IDF_SUM_BI', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN',
       'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2',
       'TD3', '+/-', 'PR', 'cluster', 'AoBFG%', 'AoBPR', 'AoB3P%', 'AoBPF',
       'AoBPTS'],
      dtype='object')

In [11]:
TextAndNBA = TextAndNBA.drop(['GP', 'FTM', 'FTA'], axis=1)

In [12]:
TextAndNBA.head()

,name,year,text,link,QA_pair_count,Sentence Count,Common Word Count,No Special Char,preprocessed_text,Word Count,...,DD2,TD3,+/-,PR,cluster,AoBFG%,AoBPR,AoB3P%,AoBPF,AoBPTS
0,Shelden Williams,2006,"Q. Going to Atlanta, any reaction? SHELDEN ...",http://www.asapsports.com/show_interview.php?i...,7,43,701,"Q. Going to Atlanta, any reaction? SHELDEN ...",q going atlanta reaction shelden williams feel...,310,...,8.0,0.0,-2.9,0.7,0,1,1,1,1,0
1,Marcus Williams,2006,"Q. How do you feel about dropping, most peopl...",http://www.asapsports.com/show_interview.php?i...,9,46,679,"Q. How do you feel about dropping, most peopl...",q feel dropping people expected go higher emot...,297,...,0.0,0.0,-0.5,0.4,0,0,0,0,0,0
2,Tyrus Thomas,2006,"Q. Talk to me about, it's been a long evening...",http://www.asapsports.com/show_interview.php?i...,5,28,417,"Q. Talk to me about, it's been a long evening...",q talk long evening finally get sitting backst...,166,...,2.0,0.0,2.1,0.7,0,1,1,0,1,0
3,Cedric Simmons,2006,Q. I guess it must be exciting to go to New O...,http://www.asapsports.com/show_interview.php?i...,2,8,126,Q. I guess it must be exciting to go to New O...,q guess must exciting go new orleans family pr...,58,...,0.0,0.0,-2.3,0.6,1,0,1,0,0,0
4,Thabo Sefolosha,2006,Q. What did you feel when you heard your name...,http://www.asapsports.com/show_interview.php?i...,3,16,215,Q. What did you feel when you heard your name...,q feel heard name knew going play allen iverso...,96,...,1.0,0.0,1.2,0.5,1,0,0,1,0,0


In [13]:
CBD = CBD.rename(columns={'player_name': "name"})
merged_df = pd.merge( TextAndNBA, CBD, on="name", how='inner')
print(len(merged_df))
print(len(TextAndNBA))
print(len(CBD))
print(merged_df.head())

274
271
1067
                name  year                                               text  \
0          John Wall  2010  Q. The Washington Wizards organization had a t...   
1    Greivis Vasquez  2010  Q. What are your feelings right now?   GREIVIS...   
2          Ekpe Udoh  2010  Q. Talk about going to Golden State, not much ...   
3        Evan Turner  2010  Q. Evan, how do you expect to fit in with the ...   
4  Patrick Patterson  2010  Q. Thoughts?   PATRICK PATTERSON: Right now ex...   

                                                link  QA_pair_count  \
0  http://www.asapsports.com/show_interview.php?i...             21   
1  http://www.asapsports.com/show_interview.php?i...             13   
2  http://www.asapsports.com/show_interview.php?i...              9   
3  http://www.asapsports.com/show_interview.php?i...             16   
4  http://www.asapsports.com/show_interview.php?i...             15   

   Sentence Count  Common Word Count  \
0             101               1

In [14]:
CBD = CBD.rename(columns={'player_name': "name"})
merg = pd.merge( TextAndNBA, CBD, on="name", how='left')
print(len(merg))
print(len(TextAndNBA))
print(len(CBD))
print(merg.head())
print(merg.columns)

385
271
1067
               name  year                                               text  \
0  Shelden Williams  2006  Q.  Going to Atlanta, any reaction?   SHELDEN ...   
1   Marcus Williams  2006  Q.  How do you feel about dropping, most peopl...   
2      Tyrus Thomas  2006  Q.  Talk to me about, it's been a long evening...   
3    Cedric Simmons  2006  Q.  I guess it must be exciting to go to New O...   
4   Thabo Sefolosha  2006  Q.  What did you feel when you heard your name...   

                                                link  QA_pair_count  \
0  http://www.asapsports.com/show_interview.php?i...              7   
1  http://www.asapsports.com/show_interview.php?i...              9   
2  http://www.asapsports.com/show_interview.php?i...              5   
3  http://www.asapsports.com/show_interview.php?i...              2   
4  http://www.asapsports.com/show_interview.php?i...              3   

   Sentence Count  Common Word Count  \
0              43                701   

In [15]:
print(merged_df.head())
print(len(merged_df))
print(merged_df["name"].head())
print(merged_df[merged_df["name"] == "Jordan Hill"]["year"])

                name  year                                               text  \
0          John Wall  2010  Q. The Washington Wizards organization had a t...   
1    Greivis Vasquez  2010  Q. What are your feelings right now?   GREIVIS...   
2          Ekpe Udoh  2010  Q. Talk about going to Golden State, not much ...   
3        Evan Turner  2010  Q. Evan, how do you expect to fit in with the ...   
4  Patrick Patterson  2010  Q. Thoughts?   PATRICK PATTERSON: Right now ex...   

                                                link  QA_pair_count  \
0  http://www.asapsports.com/show_interview.php?i...             21   
1  http://www.asapsports.com/show_interview.php?i...             13   
2  http://www.asapsports.com/show_interview.php?i...              9   
3  http://www.asapsports.com/show_interview.php?i...             16   
4  http://www.asapsports.com/show_interview.php?i...             15   

   Sentence Count  Common Word Count  \
0             101               1624   
1     

In [16]:
print(merged_df.columns)

Index(['name', 'year', 'text', 'link', 'QA_pair_count', 'Sentence Count',
       'Common Word Count', 'No Special Char', 'preprocessed_text',
       'Word Count',
       ...
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts'],
      dtype='object', length=112)


In [17]:
print(len(merged_df[merged_df['year'] >= 2019]))
print(len(merged_df.index[merged_df['year'] < 2016]))
print(len(merged_df.index[merged_df['year'] >= 2016]))

49
143
131


In [18]:
Performance = ["FG%", "PR", "3P%", "PF", "PTS"]

In [19]:
TrainIndex = list(merged_df.index[merged_df['year'] < 2016])
print(len(TrainIndex))

143


## All Embeddings

In [20]:
TextAndNBA = merged_df

In [21]:
tfidf_vectorizer = TfidfVectorizer(ngram_range =(1,1))

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf_unigrams = tfidf_vectorizer.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_unigrams.shape)

# EXTRA BELOW:

unigrams = tfidf_vectorizer.get_feature_names_out()


# create a Pandas dataframe to store the TF-IDF unigram scores
tfidf_unigram_df = pd.DataFrame(tfidf_unigrams.toarray(), columns=unigrams)

(143, 2259)


In [22]:
df2 = TfidfVectorizer(ngram_range =(1,1), min_df=2)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf2_unigrams = df2.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf2_unigrams.shape)

df3 = TfidfVectorizer(ngram_range =(1,1), min_df=3)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf3_unigrams = df3.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf3_unigrams.shape)


df4 = TfidfVectorizer(ngram_range =(1,1), min_df=4)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf4_unigrams = df4.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf4_unigrams.shape)


df5 = TfidfVectorizer(ngram_range =(1,1), min_df=5)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf5_unigrams = df5.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf5_unigrams.shape)

(143, 1872)
(143, 1537)
(143, 1290)
(143, 1103)


In [23]:
# For test data


# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf_unigramsTest = tfidf_vectorizer.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_unigramsTest.shape)

tfidf2_unigramsTest = df2.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf3_unigramsTest = df3.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf4_unigramsTest = df4.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf5_unigramsTest = df5.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

(131, 2259)


In [24]:
tfidf_vectorizer_bi = TfidfVectorizer(ngram_range =(2,2))

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf_bigrams = tfidf_vectorizer_bi.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_bigrams.shape)

# EXTRA BELOW:

# get the feature names (i.e., the unigrams)
#x_array = 
#print(tfidf_unigrams.toarray())
bigrams = tfidf_vectorizer_bi.get_feature_names_out()


# create a Pandas dataframe to store the TF-IDF unigram scores
tfidf_bigram_df = pd.DataFrame(tfidf_bigrams.toarray(), columns=bigrams)

(143, 20162)


In [25]:
dfb2 = TfidfVectorizer(ngram_range =(2,2), min_df=2)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf2_bigrams = dfb2.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf2_bigrams.shape)

dfb3 = TfidfVectorizer(ngram_range =(2,2), min_df=3)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf3_bigrams = dfb3.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf3_bigrams.shape)


dfb4 = TfidfVectorizer(ngram_range =(2,2), min_df=4)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf4_bigrams = dfb4.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf4_bigrams.shape)


dfb5 = TfidfVectorizer(ngram_range =(2,2), min_df=5)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf5_bigrams = dfb5.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf5_bigrams.shape)

(143, 10238)
(143, 4896)
(143, 2813)
(143, 1154)


In [26]:
tfidf_bigramsTest = tfidf_vectorizer_bi.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_bigramsTest.shape)


tfidf2_bigramsTest = dfb2.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf3_bigramsTest = dfb3.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf4_bigramsTest = dfb4.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf5_bigramsTest = dfb5.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

(131, 20162)


In [27]:
tfidf_vectorizer_ubi = TfidfVectorizer(ngram_range =(1,2))

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf_ubigrams = tfidf_vectorizer_ubi.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_ubigrams.shape)

# EXTRA BELOW:

# get the feature names (i.e., the unigrams)
#x_array = 
#print(tfidf_unigrams.toarray())
ubigrams = tfidf_vectorizer_ubi.get_feature_names_out()


# create a Pandas dataframe to store the TF-IDF unigram scores
tfidf_ubigram_df = pd.DataFrame(tfidf_ubigrams.toarray(), columns=ubigrams)

(143, 22421)


In [28]:
dfub2 = TfidfVectorizer(ngram_range =(1,2), min_df=2)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf2_ubigrams = dfub2.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf2_ubigrams.shape)

dfub3 = TfidfVectorizer(ngram_range =(1,2), min_df=3)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf3_ubigrams = dfub3.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf3_ubigrams.shape)


dfub4 = TfidfVectorizer(ngram_range =(1,2), min_df=4)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf4_ubigrams = dfub4.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf4_ubigrams.shape)


dfub5 = TfidfVectorizer(ngram_range =(1,2), min_df=5)

# fit and transform the text data to calculate the TF-IDF unigram scores
tfidf5_ubigrams = dfub5.fit_transform(TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf5_ubigrams.shape)

(143, 12110)
(143, 6433)
(143, 4103)
(143, 2257)


In [29]:
tfidf_ubigramsTest = tfidf_vectorizer_ubi.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])
print(tfidf_ubigramsTest.shape)


tfidf2_ubigramsTest = dfub2.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf3_ubigramsTest = dfub3.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf4_ubigramsTest = dfub4.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

tfidf5_ubigramsTest = dfub5.transform(TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'])

(131, 22421)


In [30]:
text_data = TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'].values.tolist()
tokenized_data = [text.split() for text in text_data]
VECTOR_SIZE = 60
model = Word2Vec(tokenized_data, min_count=3, vector_size=VECTOR_SIZE)
embedded_data = []
for text in tokenized_data:
    embedded_text = [model.wv[word] for word in text if word in model.wv.key_to_index]
    embedded_data.append(embedded_text)

In [31]:
lengts = np.empty(0)
for text in embedded_data:
  lengts = np.append(lengts, len(text))

print(lengts.max())
print(lengts.min())
MAXWORDS = int(lengts.max())

719.0
113.0


In [32]:
word2vec_embed = np.empty((0,MAXWORDS,VECTOR_SIZE))
for document in embedded_data:
  doc_vec = np.empty((0, VECTOR_SIZE))
  for word in document:

    doc_vec = np.vstack((doc_vec, np.reshape(word, (1, VECTOR_SIZE))))

  if doc_vec.shape[0] < MAXWORDS:
    n = MAXWORDS - doc_vec.shape[0]
    doc_vec = np.pad(doc_vec, [(0, n), (0 , 0)], mode = 'constant')
    
  elif doc_vec.shape[0] > MAXWORDS:
    doc_vec = doc_vec[:MAXWORDS, :]

  

  word2vec_embed = np.vstack((word2vec_embed, np.reshape(doc_vec, (1, MAXWORDS, VECTOR_SIZE))))

In [33]:
word2vec_embed = np.reshape(word2vec_embed, (word2vec_embed.shape[0],  word2vec_embed.shape[1] * word2vec_embed.shape[2]   ))
print(word2vec_embed.shape)

(143, 43140)


In [34]:
text_data = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'].values.tolist()
tokenized_data = [text.split() for text in text_data]
model = Word2Vec(tokenized_data, min_count=3, vector_size=VECTOR_SIZE)
embedded_data = []
for text in tokenized_data:
    embedded_text = [model.wv[word] for word in text if word in model.wv.key_to_index]
    embedded_data.append(embedded_text)


word2vec_embedTest = np.empty((0,MAXWORDS,VECTOR_SIZE))
for document in embedded_data:
  doc_vec = np.empty((0, VECTOR_SIZE))
  for word in document:

    doc_vec = np.vstack((doc_vec, np.reshape(word, (1, VECTOR_SIZE))))

  if doc_vec.shape[0] < MAXWORDS:
    n = MAXWORDS - doc_vec.shape[0]
    doc_vec = np.pad(doc_vec, [(0, n), (0 , 0)], mode = 'constant')
    
  elif doc_vec.shape[0] > MAXWORDS:
    doc_vec = doc_vec[:MAXWORDS, :]

  

  word2vec_embedTest = np.vstack((word2vec_embedTest, np.reshape(doc_vec, (1, MAXWORDS, VECTOR_SIZE))))



word2vec_embedTest = np.reshape(word2vec_embedTest, (word2vec_embedTest.shape[0],  word2vec_embedTest.shape[1] * word2vec_embedTest.shape[2]   ))
print(word2vec_embedTest.shape)

(131, 43140)


In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

## TAKES AROUND 10 MINUTES


# Load text data from pandas DataFrame
texts = TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'removed_text'].tolist()

# Tokenize text using BERT tokenizer
tokens = tokenizer.batch_encode_plus(texts,
                                      add_special_tokens=True,
                                      padding=True,
                                      truncation=True,
                                      return_tensors='pt')

# Generate BERT embeddings for the input text
with torch.no_grad():
    embeddings = model(tokens['input_ids'], attention_mask=tokens['attention_mask'])[0]

# Extract the embeddings for the first token (the [CLS] token)
bert_embeddings = embeddings[:, 0, :].numpy()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
texts = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), 'removed_text'].tolist()

# Tokenize text using BERT tokenizer
tokens = tokenizer.batch_encode_plus(texts,
                                      add_special_tokens=True,
                                      padding=True,
                                      truncation=True,
                                      return_tensors='pt')

# Generate BERT embeddings for the input text
with torch.no_grad():
    embeddings = model(tokens['input_ids'], attention_mask=tokens['attention_mask'])[0]

# Extract the embeddings for the first token (the [CLS] token)
bert_embeddingsTest = embeddings[:, 0, :].numpy()
print(bert_embeddingsTest.shape)

(131, 768)


In [37]:
ALL_DATA = {"unigram_tfidf": tfidf_unigrams, "unigram_tfidf2": tfidf2_unigrams, "unigram_tfidf3": tfidf3_unigrams, "unigram_tfidf4": tfidf4_unigrams, "unigram_tfidf5": tfidf5_unigrams,
            "bigram_tfidf": tfidf_bigrams, "bigram_tfidf2": tfidf2_bigrams, "bigram_tfidf3": tfidf3_bigrams, "bigram_tfidf4": tfidf4_bigrams, "bigram_tfidf5": tfidf5_bigrams,
            "ubigram_tfidf": tfidf_ubigrams, "ubigram_tfidf2": tfidf2_ubigrams, "ubigram_tfidf3": tfidf3_ubigrams, "ubigram_tfidf4": tfidf4_ubigrams, "ubigram_tfidf5": tfidf5_ubigrams,
            "word2vec": word2vec_embed, "bert_embeddings": bert_embeddings}
ALL_TEST = {"unigram_tfidf": tfidf_unigramsTest, "unigram_tfidf2": tfidf2_unigramsTest, "unigram_tfidf3": tfidf3_unigramsTest, "unigram_tfidf4": tfidf4_unigramsTest, "unigram_tfidf5": tfidf5_unigramsTest,
            "bigram_tfidf": tfidf_bigramsTest, "bigram_tfidf2": tfidf2_bigramsTest, "bigram_tfidf3": tfidf3_bigramsTest, "bigram_tfidf4": tfidf4_bigramsTest, "bigram_tfidf5": tfidf5_bigramsTest,
            "ubigram_tfidf": tfidf_ubigramsTest, "ubigram_tfidf2": tfidf2_ubigramsTest, "ubigram_tfidf3": tfidf3_ubigramsTest, "ubigram_tfidf4": tfidf4_ubigramsTest, "ubigram_tfidf5": tfidf5_ubigramsTest,
            "word2vec": word2vec_embedTest, "bert_embeddings": bert_embeddingsTest}

# T Models

## Regression Models

In [38]:
TRegModels = {}
RegEmbedding = ["ubigram_tfidf4", "ubigram_tfidf4", "unigram_tfidf3", "unigram_tfidf5", "unigram_tfidf"]
Estimators = [25, 100, 100, 50, 75]
Features = ['log2', 'special', 'sqrt', 'sqrt', 'sqrt']


for i, perf in enumerate(Performance):
  YTDATA = TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), f'{perf}'].values

  TRegModels[perf] = None
  XTData = ALL_DATA[RegEmbedding[i]]
  num_estimator = Estimators[i]
  num_feature = Features[i]
  if num_feature == "special":
      num_feature = int(XTData.shape[1] / 3)
  



  sample_tree = DecisionTreeRegressor(max_features=num_feature)
  adareg = AdaBoostRegressor(estimator = sample_tree, n_estimators= num_estimator)
  scores = cross_validate(adareg, XTData, YTDATA, cv=10, return_estimator=True)
  mean_performance = scores['test_score'].mean()
  print(f"mean_performance for {perf}: {mean_performance}")

  bestModelIndex = np.argmax(scores['test_score'])
  

  TRegModels[perf] = scores["estimator"][bestModelIndex]

mean_performance for FG%: -0.35954901940755024
mean_performance for PR: -0.528111677551793
mean_performance for 3P%: -0.4298235161267579
mean_performance for PF: -0.2927741114160488
mean_performance for PTS: 0.06447630802736642


## Classification Models

In [57]:
perf_bests = []
perf_models = []
perf_model_names = []
num_estimators = list(range(0, 111, 5))
num_estimators[0] = 1

best_estimators = []
best_features = []


num_features = ["sqrt", "log2", "special", "auto"]

for perf in Performance:
  y = TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].values
  best_performance = 0
  best_scores = None
  best_embedding = ""
  best_esti = 0
  best_feature = None

  for num_feature in num_features:
    for num_estimator in num_estimators:
      for embeddingType in ALL_DATA.keys():
        X = ALL_DATA[embeddingType]

        if num_feature == "special":
          num_feature = int(X.shape[1] / 3)

        sample_tree = DecisionTreeClassifier(max_features=num_feature)
        adareg = AdaBoostClassifier(estimator = sample_tree, n_estimators= num_estimator)
        scores = cross_validate(adareg, X, y, cv=10, return_estimator=True)
        mean_performance = scores['test_score'].mean()
        if mean_performance > best_performance:
          best_performance = mean_performance
          best_scores = scores
          best_embedding = embeddingType
          best_esti = num_estimator
          best_feature = num_feature
  print(f"Best performance for AoB{perf}: {best_performance}")
  print(f"Used model {best_embedding}")
  print(f"Best Estimator {best_esti}")
  print(f"Best Feature {best_feature}")


  best_features.append(best_feature)
  best_estimators.append(best_esti)
  bestModelIndex = np.argmax(best_scores['test_score'])
  perf_bests.append(best_performance)
  perf_models.append(best_scores["estimator"][bestModelIndex])
  perf_model_names.append(best_embedding)

Best performance for AoBFG%: 0.7833333333333334
Used model ubigram_tfidf2
Best Estimator 35
Best Feature log2
Best performance for AoBPR: 0.8138095238095238
Used model unigram_tfidf3
Best Estimator 75
Best Feature auto
Best performance for AoB3P%: 0.7490476190476192
Used model bigram_tfidf5
Best Estimator 80
Best Feature auto
Best performance for AoBPF: 0.7842857142857143
Used model unigram_tfidf5
Best Estimator 70
Best Feature auto
Best performance for AoBPTS: 0.7857142857142857
Used model bigram_tfidf3
Best Estimator 20
Best Feature log2


In [58]:
TClassModels = {}
ClassEmbedding = {}

for i, perf in enumerate(Performance):
  TClassModels[perf] = perf_models[i]
  ClassEmbedding[perf] = perf_model_names[i]

# ClassEmbedding = ["word2vec", "ubigram_tfidf", "bigram_tfidf5", "unigram_tfidf", "bigram_tfidf2"]
# Estimators = [70, 35, 20, 110, 85]
# Features = ['auto', 'sqrt', 'auto', 'auto', 'auto']


# TClassModels = {}


# for i, perf in enumerate(Performance):
#   YTDATA = TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].values

#   TClassModels[perf] = None
#   XTData = ALL_DATA[ClassEmbedding[i]]
#   num_estimator = Estimators[i]
#   num_feature = Features[i]
#   if num_feature == "special":
#       num_feature = int(XTData.shape[1] / 3)
  



#   sample_tree = DecisionTreeClassifier(max_features=num_feature)
#   adareg = AdaBoostClassifier(estimator = sample_tree, n_estimators= num_estimator)
#   scores = cross_validate(adareg, XTData, YTDATA, cv=10, return_estimator=True)
#   mean_performance = scores['test_score'].mean()
#   print(f"mean_performance for {perf}: {mean_performance}")

#   bestModelIndex = np.argmax(scores['test_score'])
  

#   TClassModels[perf] = scores["estimator"][bestModelIndex]

In [ ]:
cperf = "PTS"
feature_importance = np.mean([
    tree.feature_importances_ for tree in TClassModels[cperf].estimators_
], axis=0)

# Create a dictionary of feature importance values
print(ALL_DATA[ClassEmbedding[cperf]].shape)
print(feature_importance)
print(feature_importance.shape)
#print(tfidf_ubigram_df)


bigrams3 = dfb3.get_feature_names_out()


# create a Pandas dataframe to store the TF-IDF unigram scores
tfidf_bigram_df3 = pd.DataFrame(tfidf3_bigrams.toarray(), columns=bigrams3)

ThingImLookingAt = tfidf_bigram_df3


# Create a dictionary of feature importance values
feature_importance_dict = dict(zip(ThingImLookingAt.columns, feature_importance))

# Print the feature importance values
for feature, importance in feature_importance_dict.items():
    print(f"Feature '{feature}': Importance = {importance:.4f}")

# Print the feature importance values
# for feature, importance in feature_importance_dict.items():
#     print(f"Feature '{feature}': Importance = {importance:.4f}")

In [80]:

sorted_items = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:10]


# Extract the keys and values from the sorted items
keys = [item[0] for item in sorted_items]
values = [item[1] for item in sorted_items]

# Print the keys and values
print("Keys for the top 10 highest values:")
print(keys)
print("Values for the top 10 highest values:")
print(values)

for i, key in enumerate(keys):
  print(f"{key}: {values[i]}")

Keys for the top 10 highest values:
['keep working', 'sixers team', 'ive got', 'im bigger', 'much know', 'hes giving', 'type player', 'earn spot', 'player think', 'game basketball']
Values for the top 10 highest values:
[0.08041959252603259, 0.060436438349978354, 0.057198772009801474, 0.057129853289545486, 0.05421448825276794, 0.051457819358559496, 0.04730911983032914, 0.04505630460031266, 0.04439827998395065, 0.0438840251819793]
keep working: 0.08041959252603259
sixers team: 0.060436438349978354
ive got: 0.057198772009801474
im bigger: 0.057129853289545486
much know: 0.05421448825276794
hes giving: 0.051457819358559496
type player: 0.04730911983032914
earn spot: 0.04505630460031266
player think: 0.04439827998395065
game basketball: 0.0438840251819793


In [73]:
from sklearn.tree import export_text
decision_tree = TClassModels[f"{cperf}"].estimators_[0]
decision_rules = export_text(decision_tree, feature_names=ThingImLookingAt.columns.tolist())

# Print the decision rules or splits
print("Decision rules or splits for the first decision tree:")
print(decision_rules)

Decision rules or splits for the first decision tree:
|--- compared <= 0.04
|   |--- pick draft <= 0.01
|   |   |--- playoff team <= 0.04
|   |   |   |--- playing four <= 0.02
|   |   |   |   |--- guys team <= 0.02
|   |   |   |   |   |--- order <= 0.02
|   |   |   |   |   |   |--- taking fifth <= 0.03
|   |   |   |   |   |   |   |--- chicago working <= 0.02
|   |   |   |   |   |   |   |   |--- lot conversation <= 0.02
|   |   |   |   |   |   |   |   |   |--- combo guard <= 0.01
|   |   |   |   |   |   |   |   |   |   |--- care <= 0.04
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 22
|   |   |   |   |   |   |   |   |   |   |--- care >  0.04
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |   |--- combo guard >  0.01
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- lot conversation >  0.02
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- chic

In [74]:
from sklearn.tree import export_graphviz

# Creates dot file named tree.dot
export_graphviz(
            decision_tree,
            out_file =  "myTreeName.dot",
            feature_names = list(ThingImLookingAt.columns),
            filled = True,
            rounded = True)


# M Model

## Regression Models

In [ ]:
# Using college data to train and nba data as target
def subset_model_PR(college_file, nba_file, k=9, test_size=0.2, random_state=42, new_data=None):
    # Load the data from the CSV files
    college_data = pd.read_csv(college_file)
    nba_data = pd.read_csv(nba_file)

    # Merge college_data with nba_data to get target variable (PR)

    #nba_data = nba_data[nba_data["PLAYER"] ==   TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), 'name']   ]

    perf = "PTS"

    merged_data = pd.merge(college_data, nba_data[['PLAYER',perf]], left_on='player_name', right_on='PLAYER')


    #TextAndNBA.loc[TextAndNBA.index.isin(TrainIndex), Xlist]

    # Drop any rows with missing target variable values
    merged_data = merged_data.dropna(subset=[perf])
    print(len(merged_data))
    merged_data = merged_data[merged_data['year'] < 2016]
    print(len(merged_data))
    
    merged_data = merged_data[merged_data['player_name'].isin(merged_df['name'])]

    #print(len(merged_data))
    #print(merged_data['year'].unique())
    # Split the data into features (X) and target (y)
    # Modify this line to drop only columns not relevant for training
    X = merged_data.drop(['player_name', 'PLAYER', perf, 'team', 'conf', 'Unnamed: 64', 'Unnamed: 65', 'yr', 'ht', 'type'], axis=1)
    #Xlist = list(X.columns)
    y = merged_data[perf]

    # The rest of your function can remain the same
    # Use feature selection to identify the most important features
    selector = SelectKBest(score_func=f_regression, k=k)

    # Impute missing values using mean strategy
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

    # Create a pipeline to handle feature selection and imputation
    pipeline = Pipeline([('imputer', imputer), ('selector', selector)])

    # Fit the pipeline on the data and transform it
    X_selected = pipeline.fit_transform(X, y)
    #print(X_selected)
    #print(type(X_selected))
    #print(X_selected.shape)
    # Get the indices and names of the most important features
    indices = selector.get_support(indices=True)
    feature_names = X.columns[indices]
    print(list(feature_names))
    # Split the subset of data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=test_size, random_state=random_state)

    # Train a machine learning model on the subset of data
    model = RandomForestRegressor(random_state=random_state)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(score)
    return model, list(feature_names)


PRModel, Xlist = subset_model_PR("/content/drive/MyDrive/448 - Project/CollegeBasketballPlayers2009-2021.csv", "/content/drive/MyDrive/448 - Project/ALL YEARS 2009 - 2022.csv", k=9)

<ipython-input-38-3fe0922937b4>:4: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  college_data = pd.read_csv(college_file)


1255
849
['usg', 'ORB_per', 'AST_per', 'blk_per', 'stl_per', 'ast/tov', 'pick', 'dbpm', 'ast']
0.32669363219783853


In [ ]:
def subset_model_all_features(college_file, nba_file, features, k=9, test_year=2016, test_size=0.2, random_state=42, new_data=None):
    # Load the data from the CSV files
    college_data = pd.read_csv(college_file)
    nba_data = pd.read_csv(nba_file)

    Models = {}
    FeatNames = {}
    for feat in features:
        # Find the latest year in college_data that is before the test_year
        year = college_data['year'].max()
        while year >= test_year:
            year -= 1

        # Merge college_data with nba_data to get target variable (PR)
        merged_data = pd.merge(college_data, nba_data[['PLAYER', feat]], left_on='player_name', right_on='PLAYER')

        # Drop any rows with missing target variable values
        merged_data = merged_data.dropna(subset=[feat])

        merged_data = merged_data[merged_data['year'] < 2016]
        merged_data = merged_data[merged_data['player_name'].isin(merged_df['name'])]


        # Split the data into features (X) and target (y)
        # Modify this line to drop only columns not relevant for training
        X = merged_data.drop(['player_name', 'PLAYER', feat, 'team', 'conf', 'Unnamed: 64', 'Unnamed: 65', 'yr', 'ht', 'type', 'year'], axis=1)
        y = merged_data[feat]
        print(X.shape)

        feature_names = X.columns
        print(list(feature_names))
        
        # The rest of your function can remain the same
        # Use feature selection to identify the most important features
        selector = SelectKBest(score_func=f_regression, k=k)

        # Impute missing values using mean strategy
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

        # Create a pipeline to handle feature selection and imputation
        pipeline = Pipeline([('imputer', imputer), ('selector', selector)])

        # Fit the pipeline on the data and transform it
        X_selected = pipeline.fit_transform(X, y)

        # Get the indices and names of the most important features
        indices = selector.get_support(indices=True)
        feature_names = X.columns[indices]
        print(list(feature_names))
        # Split the subset of data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=test_size, random_state=random_state)

        # Train a machine learning model on the subset of data
        model = RandomForestRegressor(random_state=random_state)
        model.fit(X_train, y_train)

        # Evaluate the model on the testing set
        score = model.score(X_test, y_test)
        Models[feat] = model
        FeatNames[feat] = list(feature_names)
        print(f"{feat}: {score}")
    return Models, FeatNames
MRegModels, Xlists = subset_model_all_features("/content/drive/MyDrive/448 - Project/CollegeBasketballPlayers2009-2021.csv", "/content/drive/MyDrive/448 - Project/ALL YEARS 2009 - 2022.csv", Performance, k=9)

<ipython-input-70-6672c7d33533>:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  college_data = pd.read_csv(college_file)


(202, 57)
['TPM', 'TPA', 'TP_per', 'blk_per', 'dunksmade', 'dunksmiss+dunksmade', 'pick', 'dbpm', 'blk']
FG%: 0.6585051654443879
(202, 57)
['ORB_per', 'DRB_per', 'AST_per', 'ast/tov', 'dunksmade', 'dunksmiss+dunksmade', 'oreb', 'ast', 'stl']
PR: 0.26226633196721316
(202, 57)
['ORB_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'ftr', 'dunksmade', 'dunksmiss+dunksmade', 'blk']
3P%: -0.25461163658287855
(202, 57)
['DRB_per', 'TPM', 'TPA', 'pfr', 'dunksmade', 'dunksmiss+dunksmade', 'drtg', 'adrtg', 'dbpm']
PF: -0.11853074191912216
(202, 57)
['usg', 'ORB_per', 'AST_per', 'blk_per', 'stl_per', 'ast/tov', 'pick', 'dbpm', 'ast']
PTS: 0.32669363219783853


In [ ]:
print(Xlists)
print(MRegModels)

{'FG%': ['TPM', 'TPA', 'TP_per', 'blk_per', 'dunksmade', 'dunksmiss+dunksmade', 'pick', 'dbpm', 'blk'], 'PR': ['ORB_per', 'DRB_per', 'AST_per', 'ast/tov', 'dunksmade', 'dunksmiss+dunksmade', 'oreb', 'ast', 'stl'], '3P%': ['ORB_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'ftr', 'dunksmade', 'dunksmiss+dunksmade', 'blk'], 'PF': ['DRB_per', 'TPM', 'TPA', 'pfr', 'dunksmade', 'dunksmiss+dunksmade', 'drtg', 'adrtg', 'dbpm'], 'PTS': ['usg', 'ORB_per', 'AST_per', 'blk_per', 'stl_per', 'ast/tov', 'pick', 'dbpm', 'ast']}
{'FG%': RandomForestRegressor(random_state=42), 'PR': RandomForestRegressor(random_state=42), '3P%': RandomForestRegressor(random_state=42), 'PF': RandomForestRegressor(random_state=42), 'PTS': RandomForestRegressor(random_state=42)}


## Classification Models

In [ ]:


def neural_network_model_classification(csv_file, nba_file, features, test_year, test_size=0.2, random_state=42):
    # Load the data from the CSV files
    college_data = pd.read_csv(csv_file, low_memory=False)
    nba_data = pd.read_csv(nba_file)

    # Initialize an empty dictionary to store accuracy scores
    scores = {}
    Models = {}
    FeatNames = {}

    for feat in features:
        # Find the latest year in college_data that is before the test_year
        year = college_data[college_data['year'] < test_year]['year'].max()

        # Merge college_data with nba_data to get target variables (above/below median)
        merged_data = pd.merge(college_data, nba_data[['PLAYER', feat]], left_on='player_name', right_on='PLAYER')

        # Drop any rows with missing target variable values
        merged_data = merged_data.dropna(subset=[feat])

        merged_data = merged_data[merged_data['year'] < 2016]
        #merged_data = merged_data[merged_data['player_name'].isin(merged_df['name'])]




        # Calculate the median of the target variable
        target_median = merged_data[feat].mean()

        # Classify the samples as above or below median
        merged_data['target'] = merged_data[feat].apply(lambda x: 1 if x >= target_median else 0)

        # Split the data into features (X) and target (y)
        X = merged_data.drop(['player_name', 'PLAYER', 'team', 'target', 'conf', 'Unnamed: 64', 'Unnamed: 65', 'year', 'yr', 'ht', 'type', feat], axis=1)
        y = merged_data['target']
        print(X.shape)


        feature_names = X.columns
        print(list(feature_names))


        # Impute missing values using mean strategy
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        X = imputer.fit_transform(X)

        # Standardize the features
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Train a neural network model on the training set
        model = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=100, random_state=random_state)
        model.fit(X_train, y_train)

        # Evaluate the model on the testing set
        score = model.score(X_test, y_test)


        Models[feat] = model
        FeatNames[feat] = list(feature_names)
        print(f"{feat}: {score}")
    return Models, FeatNames

MClassModels, XClasslist = neural_network_model_classification("/content/drive/MyDrive/448 - Project/CollegeBasketballPlayers2009-2021.csv", "/content/drive/MyDrive/448 - Project/ALL YEARS 2009 - 2022.csv", Performance, test_year=2016)

(849, 57)
['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'num', 'porpag', 'adjoe', 'pfr', 'pid', 'Rec Rank', 'ast/tov', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


FG%: 0.711764705882353
(849, 57)
['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'num', 'porpag', 'adjoe', 'pfr', 'pid', 'Rec Rank', 'ast/tov', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


PR: 0.7588235294117647
(849, 57)
['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'num', 'porpag', 'adjoe', 'pfr', 'pid', 'Rec Rank', 'ast/tov', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


3P%: 0.6352941176470588
(849, 57)
['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'num', 'porpag', 'adjoe', 'pfr', 'pid', 'Rec Rank', 'ast/tov', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


PF: 0.6764705882352942
(849, 57)
['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'num', 'porpag', 'adjoe', 'pfr', 'pid', 'Rec Rank', 'ast/tov', 'rimmade', 'rimmade+rimmiss', 'midmade', 'midmade+midmiss', 'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade', 'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']
PTS: 0.6764705882352942


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


# TM Model

## Regression Models

In [ ]:
#Performance = ['PTS']

In [ ]:
FusionModels = {}
for i, perf in enumerate(Performance):
  #print(perf)
  X_forT = ALL_TEST[RegEmbedding[i]][0:82, :]
  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), Xlists[perf]].head(82)
  
  #print(type(X_forM))
  Tmodel = TRegModels[perf]
  Mmodel = MRegModels[perf]
  y = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'{perf}'].head(82).values
  #print(f"This is y shape{y.shape}")
  Tprediction = Tmodel.predict(X_forT)
  Mprediction = Mmodel.predict(X_forM) 

  #print(type(Tprediction))
  #print(Tprediction)
  #print(Tprediction.shape)
  #print(Mprediction.shape)

  X = np.stack((Tprediction, Mprediction), axis=1) #combine Tpredict and Mpredict
  #print(X.shape)

  fusionModel = LinearRegression()
  scores = cross_validate(fusionModel, X, y, cv=10, return_estimator=True)
  mean_performance = scores['test_score'].mean()
  if perf == "FG%":
    mean_performance = scores['test_score'][~4].mean()


  print(f"mean_performance for {perf} TM Model: {mean_performance}")
  bestModelIndex = np.argmax(scores['test_score'])
  print(scores['test_score'])
  fusionModel = scores["estimator"][bestModelIndex]
  FusionModels[perf] = fusionModel
  print(f"And the Coefficients are: {fusionModel.coef_}")

  coefficients = fusionModel.coef_
  intercept = fusionModel.intercept_

  X_with_intercept = sm.add_constant(X)

  # Fit the ordinary least squares (OLS) model
  ols_model = sm.OLS(y, X_with_intercept).fit()

  # Get the p-values for the predictors (coefficients)
  p_values = ols_model.pvalues[1:]  # Exclude the intercept

  # Print the p-values
  print("P-values: ", p_values)



  #mean_performance = scores['test_score'][bestModelIndex]
  
  #print(mean_performance)



/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


mean_performance for FG% TM Model: -0.39500559259023227
[ 5.44633974e-02 -6.80309157e-01  7.85874092e-01 -1.22174091e-01
 -7.83811639e+01 -3.95005593e-01  4.72138977e-01 -3.20528961e-01
  4.72430734e-01  6.28705624e-01]
And the Coefficients are: [0.10035143 0.62075545]
P-values:  [6.60399654e-01 1.15445880e-08]
mean_performance for PR TM Model: 0.26093977283106456
[-0.10388922  0.69935114  0.71814805  0.68022378  0.12769522  0.03117099
  0.66675097  0.74294849 -0.3830565  -0.56994518]
And the Coefficients are: [-0.063155    0.84759085]
P-values:  [5.64685557e-01 2.02252185e-10]
mean_performance for 3P% TM Model: -0.38937561358377665
[-0.05401019 -0.90877148 -1.25813204 -0.08816065 -0.29045469 -0.28195998
 -0.86264384  0.18152155 -0.27149449 -0.05965034]
And the Coefficients are: [-0.32365603  0.13291538]
P-values:  [0.21167227 0.28474876]
mean_performance for PF TM Model: -0.04389885686026842
[ 0.17519134 -2.19948533  0.15737113  0.2192314   0.66408399  0.40945415
 -0.69304886  0.20123

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [2]:
from statsmodels.api import OLS
import statsmodels.api as sm
print(X.shape)

OLS(y,sm.add_constant(X)).fit().summary()

NameError: ignored

## Classification Model

In [ ]:
FusionClassModels = {}
for i, perf in enumerate(Performance):
  #print(perf)
  X_forT = ALL_TEST[ClassEmbedding[perf]][0:82, :]
  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), XClasslist[perf]].head(82)
  
  #print(type(X_forM))
  Tmodel = TClassModels[perf]
  Mmodel = MClassModels[perf]
  y = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].head(82).values
  #print(f"This is y shape{y.shape}")
  Tprediction = Tmodel.predict(X_forT)
  Mprediction = Mmodel.predict(X_forM) 

  #print(type(Tprediction))
  #print(Tprediction)
  #print(Tprediction.shape)
  #print(Mprediction.shape)

  X = np.stack((Tprediction, Mprediction), axis=1) #combine Tpredict and Mpredict
  #print(X.shape)

  fusionModel = LogisticRegression()
  scores = cross_validate(fusionModel, X, y, cv=10, return_estimator=True)
  mean_performance = scores['test_score'].mean()
  print(f"mean_performance for {perf} TM Model: {mean_performance}")
  bestModelIndex = np.argmax(scores['test_score'])
  #print(scores['test_score'])
  fusionModel = scores["estimator"][bestModelIndex]
  FusionClassModels[perf] = fusionModel
  print(f"And the Coefficients are: {fusionModel.coef_}")


  # coefficients = fusionModel.coef_.ravel()
  # intercept = fusionModel.intercept_[0]

  # X_with_intercept = sm.add_constant(X)

  # # Fit the ordinary least squares (OLS) model
  # ols_model = sm.Logit(y, X_with_intercept).fit()

  # # Get the p-values for the predictors (coefficients)
  # p_values = ols_model.pvalues[1:]  # Exclude the intercept

  # # Print the p-values
  # print("P-values: ", p_values)



/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


mean_performance for FG% TM Model: 0.586111111111111
And the Coefficients are: [[5.24698913e-01 5.99676794e-05]]
mean_performance for PR TM Model: 0.8305555555555555
And the Coefficients are: [[0.64367829 0.        ]]
mean_performance for 3P% TM Model: 0.6458333333333333
And the Coefficients are: [[-1.55162947e-01 -4.58671929e-07]]
mean_performance for PF TM Model: 0.7083333333333333
And the Coefficients are: [[-7.14976649e-01  4.62488488e-06]]
mean_performance for PTS TM Model: 0.548611111111111
And the Coefficients are: [[ 4.69146980e-01 -1.35515822e-06]]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


# Testing Model

## Linear Models

In [ ]:
for i, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'{perf}'].tail(49).values

  X_testforT = ALL_TEST[RegEmbedding[i]][82:, :]
  currentT_model = TRegModels[perf]
  Tprediction = currentT_model.predict(X_testforT)

  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), Xlists[perf]].tail(49)
  Mmodel = MRegModels[perf]
  Mprediction = Mmodel.predict(X_forM)

  print(Tprediction.shape, Mprediction.shape)

  X = np.stack((Tprediction, Mprediction), axis=1)

  score = FusionModels[perf].score(X, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score: {score}")
  #print(f"Using {embedding}, the test accuracy for AoB{perf} was: {accuracy}")

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


(49,) (49,)
FG% score: 0.31161763418288824
(49,) (49,)
PR score: 0.4367573281464068


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


(49,) (49,)
3P% score: 0.009533620311106938
(49,) (49,)
PF score: 0.1495514354086469
(49,) (49,)
PTS score: 0.1957371274607208


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [ ]:
for i, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'{perf}'].tail(49).values

  X_testforT = ALL_TEST[RegEmbedding[i]][82:, :]
  currentT_model = TRegModels[perf]
  Tprediction = currentT_model.predict(X_testforT)

  


  score = currentT_model.score(X_testforT, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score: {score}")
  #print(f"Using {embedding}, the test accuracy for AoB{perf} was: {accuracy}")

FG% score: -0.046457906895594814
PR score: -0.5688010899182558
3P% score: -0.30864453954773263
PF score: -0.24843907351460204
PTS score: -0.2859927775106399


In [ ]:
for performanceIndex, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'{perf}'].tail(49).values


  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), Xlists[perf]].tail(49)
  Mmodel = MRegModels[perf]
  score = Mmodel.score(X_forM, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score {score}")
  #print(f"Using {embedding}, the test accuracy for AoB{perf} was: {accuracy}")

FG% score 0.13915862232176957
PR score 0.3617833446866484
3P% score -0.2574008571172328
PF score 0.1904148942598186
PTS score 0.06976040126930871


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


## Classification Models

In [ ]:
for i, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].tail(49).values

  X_testforT = ALL_TEST[ClassEmbedding[perf]][82:, :]
  currentT_model = TClassModels[perf]
  Tprediction = currentT_model.predict(X_testforT)

  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), XClasslist[perf]].tail(49)
  Mmodel = MClassModels[perf]
  Mprediction = Mmodel.predict(X_forM)

  print(Tprediction.shape, Mprediction.shape)

  X = np.stack((Tprediction, Mprediction), axis=1)

  score = FusionClassModels[perf].score(X, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score: {score}")
  #print(f"Using {embedding}, the test accuracy for AoB{perf} was: {accuracy}")

(49,) (49,)
FG% score: 0.5306122448979592
(49,) (49,)
PR score: 0.9183673469387755
(49,) (49,)
3P% score: 0.7346938775510204
(49,) (49,)
PF score: 0.6530612244897959
(49,) (49,)
PTS score: 0.42857142857142855


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [ ]:
for i, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].tail(49).values

  X_testforT = ALL_TEST[ClassEmbedding[perf]][82:, :]
  currentT_model = TClassModels[perf]
  Tprediction = currentT_model.predict(X_testforT)


  score = currentT_model.score(X_testforT, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score: {score}")

FG% score: 0.5306122448979592
PR score: 0.5714285714285714
3P% score: 0.5510204081632653
PF score: 0.6530612244897959
PTS score: 0.42857142857142855


In [ ]:
for i, perf in enumerate(Performance):
  y_test = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), f'AoB{perf}'].tail(49).values


  X_forM = TextAndNBA.loc[~TextAndNBA.index.isin(TrainIndex), XClasslist[perf]].tail(49)
  Mmodel = MClassModels[perf]
  Mprediction = Mmodel.predict(X_forM)


  score = Mmodel.score(X_forM, y_test)

  #accuracy = (y_pred == y_test).mean()
  print(f"{perf} score: {score}")

FG% score: 0.5714285714285714
PR score: 0.9183673469387755
3P% score: 0.7346938775510204
PF score: 0.3469387755102041
PTS score: 0.3877551020408163


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dis